# The goal of this tutorial is use simple post processing on a rst file and export it in vtk for visualization

To use it, add PYTHONPATH to the generated python code from Protocols/packages and ANSYS_PATH to your ansys install path

1st step : load the necessary module of dpf for grpc, launch the server and create the doc

In [1]:
from ansys import dpf
from ansys.dpf.server import start_local_server

In [2]:
start_local_server()

server started at --address 10.110.2.47 --port 50052


In [3]:
doc = dpf.Operator("html_doc")
doc.inputs.output_path(r'c:/temp/doc.html')
doc.run()

2nd step : create a model with the path of the result file

In [4]:
path = r'c:/temp/cp55.rst'
model = dpf.Model(path)

3rd step : get the time freq support

In [5]:
print(model.metadata.time_freq_support)

Time/Frequency Info:
	Number of sets: 35

With complex values
 
 Cumulative      Time (s)       Loadstep     Substep   
     1             0.0             1            1      
     2             0.02            1            2      
     3             0.04            1            3      
     4             0.06            1            4      
     5             0.08            1            5      
     6             0.1             1            6      
     7             0.12            1            7      
     8             0.14            1            8      
     9             0.16            1            9      
     10            0.18            1            10     
     11            0.2             1            11     
     12           0.219            1            12     
     13           0.239            1            13     
     14           0.259            1            14     
     15           0.279            1            15     
     16           0.299            1    

4th step: get the mesh

In [7]:
print(model.metadata.meshed_region)

Meshed Region
	3820 nodes
	789 elements
	Unit: m 



In [9]:
model.metadata.meshed_region.nodes.coordinates_field.data

array([[ 6.42717166e-03,  1.08088993e-02,  1.47709405e+00],
       [-1.97611503e-03,  1.83819564e-02,  1.47709405e+00],
       [-1.25121885e-02,  1.40332062e-02,  1.47709405e+00],
       ...,
       [ 2.25827245e-19, -7.52757482e-19,  0.00000000e+00],
       [ 2.25827245e-19,  7.52757482e-19,  1.50000000e+00],
       [ 0.00000000e+00,  1.83697020e-18,  4.99999985e-01]])

5th step: look for available results in the file

In [10]:
print(model.metadata.result_info)

Static analysis
Unit system: Metric (m, kg, N, s, V, A)
Physics Type: Mecanic
Available results:
     -  displacement                  
     -  force                         
     -  element_nodal_forces          
     -  stress                        
     -  volume                        
     -  energy_stiffness_matrix       
     -  hourglass_energy              
     -  thermal_dissipation_energy    
     -  kinetic_energy                
     -  co_energy                     
     -  incremental_energy            
     -  strain                        
     -  thermal_strains               
     -  thermal_strains_eqv           
     -  swelling_strains              
     -  temperature                   



In [10]:
s = model.results.stress()

In [11]:
print(s)

DPF "S" operator
    description: Load the appropriate operator based on the data sources and read/compute element nodal component stresses. Regarding the requested location and the input mesh scoping, the result location can be Nodal/ElementalNodal/Elemental.
    Available inputs:
     o   bool_rotate_to_global (optional), expects types:
            -bool            
         help: if true the field is roated to global coordinate system (default true)
                             
     o   data_sources, expects types:
            -DataSources     
         help: if the stream is null then we need to get the file path from the data sources
                             
     o   mesh (optional), expects types:
            -MeshedRegion    
            -MeshesContainer 
                             
     o   mesh_scoping (optional), expects types:
            -ScopingsContainer
            -Scoping         
         help: mesh entities scoping, unordered_map<int, int> id to index (option

In [14]:
scop = dpf.Scoping()

In [15]:
scop.ids =[1,2]
scop.location = dpf.locations.elemental

In [17]:
s.inputs.mesh_scoping.connect(scop)
s.inputs.time_scoping.connect([7,8])

In [18]:
stress_fc = s.outputs.fields_container()

In [19]:
len(stress_fc)

2

In [20]:
stress_fc[0].data

array([[ 8.67326546e+00,  1.46283984e+00, -2.64233459e+02,
         2.13418865e+00,  1.30060291e+03, -3.36917847e+02],
       [-9.21893597e+00, -6.82619238e+00, -1.62837036e+02,
         2.91970462e-01, -9.30154053e+02, -1.14185583e+03],
       [-1.03809357e+01, -9.33482933e+00, -1.25819511e+02,
        -1.02825367e+00, -1.41818701e+03,  1.43534500e+02],
       [ 2.20658565e+00,  2.68167353e+00, -2.33509155e+02,
        -1.51050639e+00,  8.16950012e+02,  8.81713501e+02],
       [-8.26666489e+01, -8.17535858e+01, -2.00859802e+02,
         1.58695436e+00,  1.30064819e+03, -3.47917694e+02],
       [-5.41989670e+01, -5.50163956e+01, -1.29520218e+02,
         1.30539462e-01, -9.40676331e+02, -1.13931042e+03],
       [-4.35029259e+01, -4.26908073e+01, -1.01353661e+02,
        -3.43237638e-01, -1.41557544e+03,  1.48301758e+02],
       [-7.52744904e+01, -7.26176376e+01, -1.80308762e+02,
         1.11087358e+00,  8.25649475e+02,  8.72830994e+02]])

In [23]:
print(model.metadata.meshed_region.elements.element_by_id(2))

DPF Element 2
	Index: 771
	Number of nodes: 20



In [24]:
stress_fc[0].scoping.ids

[2]

In [25]:
stress_fc[1].data

array([[ 1.07772875e+01, -5.31366229e-01, -2.96828094e+02,
        -3.43447542e+00,  1.18951484e+04, -9.11835480e+01],
       [-1.11937542e+01, -1.09402399e+01, -2.29911377e+02,
         3.62882757e+00, -5.01466943e+03, -1.22094746e+04],
       [-4.77265406e+00, -9.70265484e+00, -2.06001389e+02,
        -3.30055952e+00, -1.21351133e+04, -2.49043164e+03],
       [-3.15461397e+00,  6.01372123e-01, -3.26227783e+02,
         2.07464170e+00,  4.90472461e+03,  8.97695410e+03],
       [-9.04347610e+01, -9.17243805e+01, -2.23136581e+02,
         1.84533429e+00,  1.19100225e+04, -8.91186295e+01],
       [-7.77996902e+01, -7.90817032e+01, -1.86855515e+02,
         1.89830393e-01, -5.04941602e+03, -1.22139189e+04],
       [-6.69534378e+01, -6.63915329e+01, -1.58125336e+02,
         2.04095408e-01, -1.21422568e+04, -2.49792114e+03],
       [-1.05943024e+02, -1.05290764e+02, -2.55826233e+02,
         1.88631606e+00,  4.92909766e+03,  8.97044727e+03]])

In [26]:
eqv = dpf.Operator("eqv_fc")

In [27]:
eqv.inputs.fields_container.connect(s.outputs.fields_container)

In [28]:
seqv = eqv.outputs.fields_container()

In [32]:
print(seqv[0])

DPF stress_0.119975s Field
	Location: ElementalNodal
	Unit: Pa
	1 id(s)
	data size: 8
	shape: 8



In [33]:
vtk = dpf.Operator("vtk_export")

In [36]:
vtk.inputs.file_path(r'c:/temp/file.vtk')
s.inputs.requested_location("Nodal")
scop.ids =[2,162]
s.inputs.mesh_scoping(scop)
vtk.inputs.fields1(eqv.outputs.fields_container)
vtk.run()

In [37]:
import matplotlib

## Using displacement

4 th step : create the operator reader of displacement and set it's time scoping request as the entire time freq support
connect it to a norm and to a min max and export the norm in vtk

In [12]:
displacements = model.operator("U")
timeids = list(range(1,model.metadata.time_freq_support.n_sets+1))

In [13]:
displacements.inputs.time_scoping(timeids)

In [14]:
norm = dpf.Operator("norm_fc")
norm.inputs.fields_container.connect(displacements.outputs)

In [15]:
min_max = dpf.Operator("min_max_fc")
min_max.inputs.fields_container.connect(norm.outputs)
fmin = min_max.outputs.field_min()
fmax = min_max.outputs.field_max()

In [16]:
fmin.data

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.])

In [17]:
fmax.data

array([0.        , 0.00062674, 0.0025094 , 0.00564185, 0.00999992,
       0.01552154, 0.02207871, 0.02944459, 0.03725894, 0.04499722,
       0.05195353, 0.05703912, 0.05982844, 0.05897617, 0.05358419,
       0.04310436, 0.02759782, 0.00798431, 0.0137951 , 0.03478255,
       0.05130461, 0.05942392, 0.05715204, 0.04272116, 0.01787116,
       0.01244994, 0.04062977, 0.05913066, 0.06042056, 0.0418829 ,
       0.01201879, 0.03526532, 0.05950852, 0.06077103, 0.03733769])

In [24]:
vtk = dpf.Operator("vtk_export")
vtk.inputs.mesh(model.metadata.meshed_region)
vtk.inputs.fields1(norm.outputs)
vtk.inputs.file_path(r'c:/temp/file.vtk')
vtk.run()